<a href="https://colab.research.google.com/github/doudi0101/Google_AI/blob/main/Comparaison_outils_analyse_sentiments_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis Tools

Il existe de nombreuses bibliothèques qui font l'analyse des sentiments pour vous. Imaginez ça : il suffit de prendre une phrase, de la jeter dans une bibliothèque et d'obtenir un score en retour ! Comme c'est pratique !

Cela peut aussi être **totalement irresponsable** si vous ne savez pas comment l'analyseur de sentiments a été construit. Dans cette section, nous allons voir comment l'analyse de sentiments est faite avec quelques paquets différents.

### Installation

Nous allons utiliser deux paquets de traitement du langage, NLTK et Textblob, ainsi que deux bibliothèques d'analyse de données/visualisation, matplotlib et pandas. Vous pouvez décommenter et exécuter la cellule ci-dessous si vous en avez besoin.

In [ ]:
# !pip install matplotlib pandas nltk textblob 

## Outils

### NLTK: Natural Language Tooklit

[Natural Language Toolkit](https://www.nltk.org/) est la base de beaucoup d'analyses de texte réalisées en Python. Il est vieux, terrible et lent, mais il est utilisé depuis si longtemps et fait tellement de choses qu'il est généralement utilisé par défaut lorsque les gens se lancent dans l'analyse de texte. Le petit dernier est [spaCy](https://spacy.io/) (mais il ne fait pas d'analyse de sentiments tout de suite, donc nous le laissons de côté).

Lorsque vous lancez NLTK pour la première fois, vous devez télécharger quelques jeux de données pour vous assurer qu'il sera capable de faire tout ce que vous voulez.

In [ ]:
import nltk
nltk.download('vader_lexicon')
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Pour faire de l'analyse de sentiments avec NLTK, il suffit de quelques lignes de code. Pour déterminer le sentiment, il utilise un outil appelé **VADER**.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
sia.polarity_scores("This restaurant was great, but I'm not sure if I'll go there again.")

{'compound': 0.0276, 'neg': 0.153, 'neu': 0.688, 'pos': 0.159}

Demander à `SentimentIntensityAnalyzer` le `polarity_score` nous a donné quatre valeurs dans un dictionnaire :

- **negative:** le sentiment négatif dans une phrase
- **neutre:** le sentiment neutre dans une phrase
- **positif:** le sentiment positif dans une phrase
- **compound:** le sentiment agrégé. 
    
Cela semble assez simple !

In [ ]:
text = "I just got a call from my boss - does he realise it's Saturday?"
sia.polarity_scores(text)

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

Tout comme dans la vie réelle, si vous utilisez une émoticône, on peut penser que vous êtes plus positif :

In [ ]:
text = "I just got a call from my boss - does he realise it's Saturday? :)"
sia.polarity_scores(text)

{'compound': 0.4588, 'neg': 0.0, 'neu': 0.786, 'pos': 0.214}

Mais si on échangeait l'émoticône contre un emoji ?

In [ ]:
text = "I just got a call from my boss - does he realise it's Saturday? 😊"
sia.polarity_scores(text)

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

Retour au neutre ! Pourquoi n'a-t-il pas compris l'emoji de la même façon que l'emoticon ? Eh bien, **les outils d'analyse de texte ne connaissent que les mots qu'on leur a appris,** et si VADER n'a jamais vu 😊 auparavant, il ne saura pas quoi en penser.

### TextBlob

TextBlob est construit au-dessus de NLTK, mais est infiniment plus facile à utiliser. C'est toujours lent, mais _c'est tellement tellement facile à utiliser_. 

Vous pouvez simplement donner votre phrase à TextBlob, puis demander un `.sentiment` !

In [ ]:
from textblob import TextBlob
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer

In [ ]:
blob = TextBlob("This restaurant was great, but I'm not sure if I'll go there again.")
blob.sentiment

Sentiment(polarity=0.275, subjectivity=0.8194444444444444)

Cette fois, nous obtenons une `polarité` et une `subjectivité` au lieu de tous ces scores différents, mais c'est fondamentalement la même idée.

Si vous aimez les options : il s'avère que TextBlob dispose en fait de plusieurs outils d'analyse de sentiments ! Quel plaisir ! Nous pouvons brancher un autre analyseur pour obtenir un autre résultat.

In [ ]:
blobber = Blobber(analyzer=NaiveBayesAnalyzer())

blob = blobber("This restaurant was great, but I'm not sure if I'll go there again.")
blob.sentiment

Sentiment(classification='pos', p_pos=0.5879425317005774, p_neg=0.41205746829942275)

Wow, c'est un résultat très différent. Pour comprendre pourquoi c'est si différent, nous devons parler de l'origine de ces chiffres de sentiment.

## How were they made?

La chose la plus importante à comprendre est que **le sentiment n'est jamais qu'une opinion**. Dans ce cas, c'est une opinion, oui, mais spécifiquement **l'opinion d'une machine**.

### VADER

L'analyseur d'intensité de sentiment de NLTK fonctionne en utilisant un élément appelé **VADER**, qui est une liste de mots auxquels est associé un sentiment.

|Word|Sentiment rating|
|---|---|
|tragedy|-3.4|
|rejoiced|2.0|
|disaster|-3.1|
|great|3.1|

Si vous avez plus de positifs, la phrase est plus positive. Si vous avez plus de négatifs, elle est plus négative. Il peut également prendre en compte des éléments tels que les majuscules. Vous pouvez en savoir plus sur le classificateur [ici] (http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html), ou sur l'article dont il est issu [ici] (http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf).

**Comment savent-ils ce qui est positif/négatif ? Ils ont dressé une très longue liste de mots, puis ont interrogé des internautes et les ont payés un centime pour chaque mot noté.


### TextBlob's `.sentiment`

L'analyse des sentiments de TextBlob est basée sur une bibliothèque distincte appelée [pattern] (https://www.clips.uantwerpen.be/pattern).

> Le lexique d'analyse des sentiments fourni par Pattern se concentre sur les adjectifs. Il contient des adjectifs qui apparaissent fréquemment dans les commentaires des clients, étiquetés à la main avec des valeurs de polarité et de subjectivité.

C'est le même genre de chose que VADER de NLTK, mais il examine spécifiquement les mots provenant des commentaires des clients.

**Comment savent-ils ce qui est positif/négatif ? Ils regardent (principalement) les adjectifs qui apparaissent dans les commentaires des clients et les étiquettent manuellement.

### TextBlob's `.sentiment` + NaiveBayesAnalyzer

L'autre option de TextBlob utilise un `NaiveBayesAnalyzer`, qui est une technique d'apprentissage automatique. Lorsque vous utilisez cette option avec TextBlob, le sentiment provient d'un "classificateur NLTK formé sur un corpus de critiques de films".

**Comment savent-ils ce qui est positif/négatif?** En examinant les critiques et les notes de films à l'aide de l'apprentissage automatique, l'ordinateur a _automatiquement appris_ quels mots sont associés à une note positive ou négative.

## What's this mean for me?

Lorsque vous faites de l'analyse de sentiments avec des outils comme celui-ci, vous devez vous poser quelques questions importantes : 

* D'où provient la liste des mots connus ?
* Utilise-t-il tous les mots, ou une sélection de mots ?
* D'où proviennent les scores positifs/négatifs ?

Comparons les outils que nous avons utilisés jusqu'à présent.

|technique|word source|word selection|scores|
|---|---|---|---|
|NLTK (VADER)|everywhere|hand-picked|internet people, word-by-word|
|TextBlob|product reviews|hand-picked, mostly adjectives|internet people, word-by-word|
|TextBlob + NaiveBayesAnalyzer|movie reviews|all words|automatic based on score|

Une chose importante qui devrait vous sauter aux yeux est **la différence entre les sources.**

Alors que VADER se concentre sur le contenu trouvé partout, les deux options de TextBlob sont spécifiques à certains domaines. Le [document original de VADER] (http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf) note de manière passive-agressive que VADER est efficace pour une utilisation générale, mais qu'être entraîné sur un domaine spécifique peut avoir des avantages : 

> Bien que certains algorithmes aient obtenu des résultats décents sur des données de test provenant du domaine spécifique pour lequel ils ont été expressément formés, ils ne dépassent pas de manière significative le modèle simple que nous utilisons.

En gros, ils disent : "si vous entraînez un modèle sur des mots d'un certain domaine, il sera bon pour le sentiment dans ce domaine".

## Tableau comparatif

Parce qu'ils sont construits différemment, les outils d'analyse des sentiments ne sont pas toujours d'accord. Prenons un ensemble de phrases et comparons la compréhension qu'en a chaque analyseur.

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)

df = pd.DataFrame({'content': [
    "I love love love love this kitten",
    "I hate hate hate hate this keyboard",
    "I'm not sure how I feel about toast",
    "Did you see the baseball game yesterday?",
    "The package was delivered late and the contents were broken",
    "Trashy television shows are some of my favorites",
    "I'm seeing a Kubrick film tomorrow, I hear not so great things about it.",
    "I find chirping birds irritating, but I know I'm not the only one",
]})
df

,content
0,I love love love love this kitten
1,I hate hate hate hate this keyboard
2,I'm not sure how I feel about toast
3,Did you see the baseball game yesterday?
4,The package was delivered late and the contents were broken
5,Trashy television shows are some of my favorites
6,"I'm seeing a Kubrick film tomorrow, I hear not so great things about it."
7,"I find chirping birds irritating, but I know I'm not the only one"


In [ ]:
def get_scores(content):
    blob = TextBlob(content)
    nb_blob = blobber(content)
    sia_scores = sia.polarity_scores(content)
    
    return pd.Series({
        'content': content,
        'textblob': blob.sentiment.polarity,
        'textblob_bayes': nb_blob.sentiment.p_pos - nb_blob.sentiment.p_neg,
        'nltk': sia_scores['compound'],
    })

scores = df.content.apply(get_scores)
scores.style.background_gradient(cmap='RdYlGn', axis=None, low=0.4, high=0.4)

,content,textblob,textblob_bayes,nltk
0,I love love love love this kitten,0.500000,-0.087933,0.957100
1,I hate hate hate hate this keyboard,-0.800000,-0.214151,-0.941300
2,I'm not sure how I feel about toast,-0.250000,0.394659,-0.241100
3,Did you see the baseball game yesterday?,-0.400000,0.613050,0.000000
4,The package was delivered late and the contents were broken,-0.350000,-0.574270,-0.476700
5,Trashy television shows are some of my favorites,0.000000,0.040076,0.421500
6,"I'm seeing a Kubrick film tomorrow, I hear not so great things about it.",0.800000,0.717875,-0.629600
7,"I find chirping birds irritating, but I know I'm not the only one",-0.200000,0.257148,-0.250000


Wow, ils ne sont vraiment pas d'accord les uns avec les autres ! Avec lequel êtes-vous le plus d'accord ? Est-ce que tout était "juste" ?

Alors qu'il semblait magique de pouvoir brancher une phrase dans un analyseur de sentiments et d'obtenir un résultat en retour... peut-être que les choses ne sont pas aussi magiques que nous le pensions.

## Review

**L'analyse des sentiments** consiste à juger si un morceau de texte a une émotion positive ou négative. Nous avons abordé plusieurs outils permettant de réaliser une analyse automatique des sentiments : **NLTK**, et deux techniques au sein de **TextBlob**.

Chaque outil utilise des données différentes pour déterminer ce qui est positif et négatif, et tandis que certains utilisent des **humains** pour marquer les choses comme positives ou négatives, d'autres utilisent un **apprentissage machine** automatique.

En raison de ces différences, chaque outil peut obtenir des notes de sentiment très **différentes** pour le même morceau de texte.

## Sujets de discussion

Les premières questions consistent à savoir si un analyseur peut être appliqué dans des situations autres que celles pour lesquelles il a été formé. Entre autres choses, vous devrez vous demander si la langue sur laquelle il a été formé est similaire à la langue sur laquelle vous l'utilisez.

**Il est possible d'utiliser un analyseur de sentiments construit à partir d'évaluations de produits pour vérifier le sentiment de tweets **Et de vérifier le sentiment d'évaluations de service hôtelier  ?

**Il est possible d'utiliser un analyseur de sentiments formé sur tout pour vérifier le sentiment des tweets?** Et pour vérifier le sentiment des critiques de service hôtelier  ?

**Si j'essaie de savoir si les gens aiment ou n'aiment pas ce qui se passe au cours des débats, puis-je utiliser ce genre d'outils sur les tweets ?

Nous utilisons volontairement le terme incroyablement vague de "bien", car il existe différents niveaux de confort en fonction de votre situation. Faites-vous cela pour une recherche préliminaire ? Allez-vous publier les résultats dans une revue, dans un journal, dans un rapport au travail, dans une recommandation de politique publique ? Et si je vous disais que l'idéal du "je n'utiliserais qu'un outil d'analyse de sentiments formé exactement pour mon domaine spécifique" est à la fois _rare et peu pratique ?_.

Comme nous l'avons vu dans la dernière section, **ces outils ne sont pas toujours en accord les uns avec les autres, ce qui peut être problématique.**

* Qu'est-ce qui peut les mettre en accord ou en désaccord ?
* Pensons-nous que l'un d'entre eux est le "meilleur" ?
* Pouvez-vous imaginer des moyens de tester lequel est le "meilleur" pour nos besoins ?